3. april 2016

'wir sind die coolsten wenn wir cruisen wenn wir durch die city düsen'

- Andre Lenz, Sommer 2009


Alle daten holen von theaiGames.com 

- verbindung mit urllib
- datenholen mit beautiful soup
- abspeichern in pandas dataframe mit multiindex
- exportieren als pickle zur wiederverwendung



In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:90% !important; }</style>")

In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import urllib.request
from bs4 import BeautifulSoup
import datetime

ImportError: No module named 'bs4'

In [2]:
frames = {}
gameDict = {}
fieldDict = {}
sites = [] #collection of games links
playerLinks = [] # collect links of specific player, temp
player1 = []
player2 = []
playerTags = []
desiredBots = ['Jyzaya_Ultimate']
numberOfSites = 2 #7 # how many pages of games to download
a = 15 # number of games finally extraxted if not all

In [ ]:
#%%time
#Get the Bots game links

for bot in desiredBots:
    
    for i in range(1, numberOfSites):
        print(bot + str(i))
        connectionString = 'http://theaigames.com/competitions/ultimate-tic-tac-toe/game-log/'+ bot + '/' + str(i)
        try:
            connection = urllib.request.urlopen(connectionString)
            soup = BeautifulSoup(connection, "lxml")
            for link in soup.findAll('a'): 
                link = link.get('href')
                if link.startswith('http://theaigames.com/competitions/ultimate-tic-tac-toe/games/') == True:
                    playerLinks.append(link) #write all game links to one array

            
            if len(playerLinks) == 0:
                break
            playerTagsOnSite = soup.find_all(attrs={'class': 'div-botName-gameLog'})
            playerTags = playerTags + playerTagsOnSite
            sites = sites + playerLinks
            playerLinks = []

        except:
            print("Conection failed for bot and Site: " + bot + '/' + str(i))
        finally:
            connection.close
        
            
#10, 7 sec mit 2 seiten und 1 boti



In [ ]:
# differentiate for player 1 or 2 in a list with same lenght as game link list
c = 0
for playerTag in playerTags:
    if (c%2 == 0):
        tempPlayer = playerTag.getText('div-botName-gameLog')
        tempPlayer = tempPlayer.strip()
        player1.append(tempPlayer)
    else:
        tempPlayer = playerTag.getText('div-botName-gameLog')
        tempPlayer = tempPlayer.strip()
        player2.append(tempPlayer)
    c += 1



In [12]:
# dataframe für alle games anlegen

Games = {'site': sites, 'player1': player1, 'player2':player2}
Games = pd.DataFrame(Games)

done


In [13]:
#%%time
# spieldaten holen von webseite und in 3 verschiedene Dataframes setzten


for site in sites[:a]: # pro game etwa 15 secunden
    siteData = site + '/data'
    ttt = pd.read_json(siteData, typ = 'series')
    mmm = json_normalize(ttt['settings'])
    ttt = json_normalize(ttt['states'])
    fff = ttt.field
    ttt = ttt.replace('', np.nan)
    ttt.winner =ttt.winner.fillna(method = 'bfill')
    ttt.illegalMove = ttt.illegalMove.fillna('')
    frames[site] = ttt
    gameDict[site] = mmm
    fieldDict[site] = fff

In [14]:
Field = pd.concat(fieldDict)
Field = pd.DataFrame(Field)

In [15]:
Games = pd.concat(gameDict)
Games['player1'] = player1[:a]
Games['player2'] = player2[:a]


In [16]:
#create dataframe of all moves
moves = pd.concat(frames)

In [31]:
date = datetime.datetime.now()
Games.to_pickle(str(date.date()) + '_' + str(desiredBots[0]) + '_rawGames.pkl')
moves.to_pickle(str(date.date()) + '_' + str(desiredBots[0]) + '_rawMoves.pkl')
Field.to_pickle(str(date.date()) + '_' + str(desiredBots[0]) + '_rawField.pkl')